1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.


In [1]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# Fetching the datasets
soybean_data = fetch_ucirepo('soybean')
zoo_data = fetch_ucirepo('zoo')
heart_disease_data = fetch_ucirepo('heart disease')
breast_cancer_data = fetch_ucirepo('breast cancer')
dermatology_data = fetch_ucirepo('dermatology')
mushroom_data = fetch_ucirepo('mushroom')

In [2]:
datasets = [
    ('soybean_data', 'soybean_df'),
    ('zoo_data', 'zoo_df'),
    ('heart_disease_data', 'heart_disease_df'),
    ('breast_cancer_data', 'breast_cancer_df'),
    ('dermatology_data', 'dermatology_df'),
    ('mushroom_data', 'mushroom_df')
]

# Loop over datasets to fetch and create dataframes
for dataset_name, dataframe_name in datasets:
    data = fetch_ucirepo(dataset_name.split('_')[0])
    X = data.data.features
    y = data.data.targets
    df = pd.merge(X, y, left_index=True, right_index=True)
    df = df.dropna()
    globals()[dataframe_name] = df

In [3]:
from scipy.special import logsumexp

class BernoulliMixture:
    
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
    
    def fit(self,x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # E-Step
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log_likelihood:
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        mu_place = np.where(np.max(mu, axis=0) <= 1e-15, 1e-15, mu)
        return np.tensordot(x, np.log(mu_place), (1,1))
        
    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.

In [4]:
from sklearn.preprocessing import LabelEncoder
from kmodes.kmodes import KModes
from sklearn.metrics.cluster import fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score
import numpy as np

def encode_categorical(df):
    encoder = LabelEncoder()
    encoded_df = df.copy()
    for column in df.columns:
        if df[column].dtype == 'object':
            encoded_df[column] = encoder.fit_transform(df[column])
    return encoded_df

datasets = ["soybean_df", "zoo_df", "heart_disease_df", "dermatology_df", "breast_cancer_df", "mushroom_df"]
encoded_datasets = {}

for dataset_name in datasets:
    df = globals()[dataset_name].copy()
    encoded_datasets[dataset_name] = encode_categorical(df)

# perform clustering and evaluate performance
def evaluate_clustering(dataset_name, algorithm, **kwargs):
    dataset = encoded_datasets[dataset_name]
    X = dataset.iloc[:, :-1]
    true_labels = dataset.iloc[:, -1]
    
    if algorithm == 'BernoulliMixture':
        model = BernoulliMixture(**kwargs)
        model.fit(X)
        labels = model.predict(X)
    elif algorithm == 'KModes':
        km = KModes(**kwargs)
        labels = km.fit_predict(X)
    else:
        raise ValueError("Algorithm not supported.")
    
    fmi, ari, nmi = None, None, None
    
    if true_labels is not None:
        fmi = fowlkes_mallows_score(true_labels, labels)
        ari = adjusted_rand_score(true_labels, labels)
        nmi = normalized_mutual_info_score(true_labels, labels)
    
    return fmi, ari, nmi

results = {}
algorithms = ['BernoulliMixture', 'KModes']
for dataset_name in datasets:
    results[dataset_name] = {}
    for algorithm in algorithms:
        if algorithm == 'BernoulliMixture':
            kwargs = {'n_components': 2, 'max_iter': 100}
        elif algorithm == 'KModes':
            kwargs = {'n_clusters': 2, 'max_iter': 100}
        fmi, ari, nmi = evaluate_clustering(dataset_name, algorithm, **kwargs)
        results[dataset_name][algorithm] = {'FMI': fmi, 'ARI': ari, 'NMI': nmi}

C:\Users\barri\AppData\Local\Temp\ipykernel_13848\2876627655.py:72: RuntimeWarning: invalid value encountered in log
  return np.tensordot(x, np.log(mu_place), (1,1))
C:\Users\barri\AppData\Local\Temp\ipykernel_13848\2876627655.py:72: RuntimeWarning: divide by zero encountered in log
  return np.tensordot(x, np.log(mu_place), (1,1))
C:\Users\barri\AppData\Local\Temp\ipykernel_13848\2876627655.py:72: RuntimeWarning: invalid value encountered in log
  return np.tensordot(x, np.log(mu_place), (1,1))
C:\Users\barri\AppData\Local\Temp\ipykernel_13848\2876627655.py:72: RuntimeWarning: invalid value encountered in log
  return np.tensordot(x, np.log(mu_place), (1,1))
C:\Users\barri\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\special\_logsumexp.py:116: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)


-79.52632309808814
60.176732851888666
-1454.5311561165418
-215.85960219179984
-30.014906911867673
-122.17863088271564


C:\Users\barri\AppData\Local\Temp\ipykernel_13848\2876627655.py:72: RuntimeWarning: invalid value encountered in log
  return np.tensordot(x, np.log(mu_place), (1,1))
C:\Users\barri\AppData\Local\Temp\ipykernel_13848\2876627655.py:72: RuntimeWarning: invalid value encountered in log
  return np.tensordot(x, np.log(mu_place), (1,1))
C:\Users\barri\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\special\_logsumexp.py:116: RuntimeWarning: overflow encountered in exp
  tmp = np.exp(a - a_max)


In [5]:
data = []

for dataset_name, algorithms in results.items():
    for algorithm, metrics in algorithms.items():
        data.append([dataset_name, algorithm, metrics['FMI'], metrics['ARI'], metrics['NMI']])

results_df = pd.DataFrame(data, columns=['Dataset', 'Algorithm', 'FMI', 'ARI', 'NMI'])
results_df

,Dataset,Algorithm,FMI,ARI,NMI
0,soybean_df,BernoulliMixture,0.312319,0.084182,0.235825
1,soybean_df,KModes,0.355471,0.098012,0.303596
2,zoo_df,BernoulliMixture,0.674122,0.447982,0.579111
3,zoo_df,KModes,0.674122,0.447982,0.579111
4,heart_disease_df,BernoulliMixture,0.588821,0.000985,0.007023
5,heart_disease_df,KModes,0.603760,0.296890,0.207837
6,dermatology_df,BernoulliMixture,0.444244,0.000374,0.015825
7,dermatology_df,KModes,0.479534,0.154536,0.362217
8,breast_cancer_df,BernoulliMixture,0.719223,0.025469,0.004773
9,breast_cancer_df,KModes,0.569791,0.016710,0.004097


7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes

In the provided results, the FMI consistently outperforms the ARI and NMI across various datasets and clustering algorithms. FMI measures the similarity of two clusterings by considering pairwise agreements between elements in the same clusters. Therefore, it tends to produce higher values when there is strong agreement between pairs of samples in the same clusters. On the other hand, ARI and NMI generally show lower values compared to FMI. ARI quantifies the similarity between two clusterings by comparing the agreement of pairs of samples in the true and predicted clusters, adjusted for chance, while NMI measures the mutual information shared between the true and predicted labels, normalized by entropy terms. However, they may produce lower values in scenarios where the number of clusters varies, ground truth labels are unbalanced, or there are challenges such as noisy data or overlapping clusters. Additionally, the performance of clustering algorithms varies across datasets, with KModes often performing better than Bernoulli Mixture SEM, possibly due to dataset characteristics and algorithm suitability. Overall, the consistent higher FMI values underscore its robustness in evaluating clustering performance, especially in scenarios where pairwise comparisons of clusters are meaningful, despite the challenges that ARI and NMI may face in accurately capturing the clustering structure.